# Software Nanopublication Generator

Creates software description nanopublications from a JSON configuration file.

**Template:** [Software Description Template](https://w3id.org/np/RABBzVTxosLGT4YBCfdfNd6LyuOOTe2EVOTtWJMyOoZHk)

## Software Nanopublications
Document research software with:
- Title and description
- Repository/maintainer link
- Related publications (via cito:supports)
- Related resources (datasets, websites, other nanopubs)

In [ ]:
import json
import sys
import re
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from nanopub_utils import (
    NanopubGenerator, load_config, save_nanopub,
    make_literal, PREFIXES
)

In [ ]:
# Software-specific constants
SOFTWARE_TEMPLATE = "https://w3id.org/np/RABBzVTxosLGT4YBCfdfNd6LyuOOTe2EVOTtWJMyOoZHk"
DCMITYPE_SOFTWARE = "http://purl.org/dc/dcmitype/Software"
DCT_TITLE = "http://purl.org/dc/terms/title"
DCT_LICENSE = "http://purl.org/dc/terms/license"
CITO_SUPPORTS = "http://purl.org/spar/cito/supports"
SCHEMA_MAINTAINER = "https://schema.org/maintainer"
SKOS_RELATED = "http://www.w3.org/2004/02/skos/core#related"

# Software uses 3 pubinfo templates
SOFTWARE_PUBINFO_TEMPLATES = [
    "https://w3id.org/np/RA0J4vUn_dekg-U1kK3AOEt02p9mT2WO03uGxLDec1jLw",
    "https://w3id.org/np/RAoTD7udB2KtUuOuAe74tJi1t3VzK0DyWS7rYVAq1GRvw",
    "https://w3id.org/np/RAukAcWHRDlkqxk7H2XNSegc1WnHI569INvNr-xdptDGI"
]

def slugify(text):
    """Convert text to URL-safe slug."""
    text = text.lower()
    text = re.sub(r'[^\w\s-]', '', text)
    text = re.sub(r'[\s_]+', '-', text)
    return text.strip('-')

class SoftwareNanopubGenerator(NanopubGenerator):
    """Generator for software nanopublications matching Nanodash format."""
    
    # Override default pubinfo templates for software
    PUBINFO_TEMPLATES = SOFTWARE_PUBINFO_TEMPLATES
    
    def __init__(self, config: dict, nanopub_config: dict):
        # Merge metadata with individual nanopub config
        merged_config = {
            **config.get('metadata', {}),
            **nanopub_config,
            'template_uri': SOFTWARE_TEMPLATE
        }
        super().__init__(merged_config)
        
        # Add nanopub type for Software
        self.add_nanopub_type(DCMITYPE_SOFTWARE)
    
    def generate_assertion(self) -> str:
        """Generate the software assertion graph."""
        title = self.config['title']
        repository = self.config.get('repository_uri')
        license_uri = self.config.get('license_uri')
        related_publications = self.config.get('related_publications', [])
        related_resources = self.config.get('related_resources', [])
        
        # Create local slug-based URI for the software (like Nanodash does)
        software_slug = slugify(title)
        software_local = f"{self.sub_prefix}:{software_slug}"
        
        # Set label
        self.config['label'] = f"Software: {title}"
        
        lines = [f'{self.sub_prefix}:assertion {{']
        
        # Collect all predicates
        predicates = [f'    a <{DCMITYPE_SOFTWARE}>']
        
        # License (optional)
        if license_uri:
            predicates.append(f'    <{DCT_LICENSE}> <{license_uri}>')
        
        # Title (required)
        predicates.append(f'    <{DCT_TITLE}> {make_literal(title)}')
        
        # Related publications (cito:supports)
        for pub in related_publications:
            pub_uri = pub if pub.startswith('http') else f'https://doi.org/{pub}'
            predicates.append(f'    <{CITO_SUPPORTS}> <{pub_uri}>')
        
        # Related resources (skos:related) - datasets, websites, other nanopubs
        for resource in related_resources:
            predicates.append(f'    <{SKOS_RELATED}> <{resource}>')
        
        # Repository/maintainer (optional)
        if repository:
            predicates.append(f'    <{SCHEMA_MAINTAINER}> <{repository}>')
        
        # Format the software with all predicates
        lines.append(f'  {software_local}')
        for i, pred in enumerate(predicates):
            if i < len(predicates) - 1:
                lines.append(f'{pred};')
            else:
                lines.append(f'{pred} .')
        
        lines.append('}')
        return '\n'.join(lines)

In [ ]:
# Configuration
CONFIG_FILE = "../config/vbae208_software.json"  # Change this to use different config
OUTPUT_DIR = "../output/software"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# Load configuration
config = load_config(CONFIG_FILE)

print(f"Source paper: {config['metadata']['source_paper']['title']}")
print(f"DOI: {config['metadata']['source_paper']['doi']}")
print(f"Number of software nanopublications to generate: {len(config['nanopublications'])}")
print()

for i, np_config in enumerate(config['nanopublications'], 1):
    print(f"{i}. {np_config['title']}")

In [ ]:
# Generate nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create generator
    generator = SoftwareNanopubGenerator(config, np_config)
    
    # Generate nanopub content
    nanopub_content = generator.generate()
    
    # Save to file
    output_file = f"{OUTPUT_DIR}/{np_config['id']}.trig"
    save_nanopub(nanopub_content, output_file)
    generated_files.append(output_file)
    
    print(f"Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

In [ ]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

## JSON Config Structure

```json
{
  "metadata": {
    "source_paper": {
      "title": "Paper Title",
      "doi": "10.xxxx/xxxxx"
    },
    "creator_orcid": "0000-0000-0000-0000",
    "creator_name": "Your Name"
  },
  "nanopublications": [
    {
      "id": "software_myapp",
      "title": "My Software Tool",
      "repository_uri": "https://github.com/user/repo",
      "license_uri": "https://w3id.org/np/...",
      "related_publications": ["10.xxxx/xxxxx"],
      "related_resources": [
        "https://doi.org/10.xxxx/dataset",
        "https://example.com/website"
      ]
    }
  ]
}
```

### Fields

| Field | Required | Description |
|-------|----------|-------------|
| `id` | Yes | Unique identifier for the output file |
| `title` | Yes | Software name/title |
| `repository_uri` | No | GitHub/GitLab repository URL |
| `license_uri` | No | Nanopub URI for the license |
| `related_publications` | No | DOIs of papers using/describing the software |
| `related_resources` | No | Related datasets, websites, nanopubs |

## Next Steps

1. Review the generated `.trig` files in the output directory
2. Use **sign_and_publish_nanopub.ipynb** to sign and publish
3. To use with different software, create a new JSON config file and update `CONFIG_FILE`